In [ ]:
import keras 
pat_tent = '/home/silvia/Documents/GitHub/primoprogetto/Codici/Tentativi/'
tentativo = 52
model = keras.models.load_model(pat_tent+str(tentativo)+'/Tentativo_'+str(tentativo)+'.hdf5')
model2 = keras.models.load_model(pat_tent+str(tentativo)+'/Tentativo_'+str(tentativo)+'.hdf5')


In [18]:
import numpy as np
import tensorflow as tf
def my_loss(y_pred, y_true):
    # Attento, keras ammette solo questi 2 argomenti
    # Posso metterne altri seguendo le indicazioni in https://www.kdnuggets.com/2019/04/advanced-keras-constructing-complex-custom-losses-metrics.html
    return -(y_true*tf.math.log(y_pred)+ (1-y_true)* tf.math.log(1-y_pred))
model2.compile(loss=my_loss)

In [13]:
# Dataset, non importa
import sys
sys.path.append('/home/silvia/Documents/GitHub/primoprogetto/Codici/')  # PER importare cose che non sono nella stessa directory
from Classe_sismogramma_v3 import ClasseDataset
import numpy as np

hdf5_predicting = '/home/silvia/Desktop/Data/SCSN(Ross)/Ross_test_polarity_Normalizzate20_New1-1_data.hdf5'
csv_predicting =  '/home/silvia/Desktop/Data/SCSN(Ross)/Ross_test_polarity_Normalizzate20_New1-1_metadata.csv'

Data_predicting = ClasseDataset()
Data_predicting.leggi_custom_dataset(hdf5_predicting, csv_predicting)
sample_train = len(Data_predicting.sismogramma)

lung = len(Data_predicting.sismogramma[0])
semi_amp = 80
pat_tent = '/home/silvia/Documents/GitHub/primoprogetto/Codici/Tentativi/'
shiftRoss = 1
tentativo = 52
salva_predizioni = False
nome_predizione = "/Predizioni_Ross_shift+1"

xtest = np.zeros((sample_train, semi_amp * 2))
for k in range(sample_train):
    xtest[k] = Data_predicting.sismogramma[k][lung // 2 - semi_amp + shiftRoss:lung // 2 + semi_amp + shiftRoss]
y_test_true = np.array([Data_predicting.metadata["trace_polarity"][_] == "positive" for _ in range(sample_train)])
y_test_true = y_test_true + 0
y_test_true = np.array(y_test_true) + 0.0

In [14]:

evaluation_results = model.evaluate(xtest, y_test_true, batch_size=2048)
# Print the evaluation metrics
print("Evaluation Metrics:")
for i, metric_name in enumerate(model.metrics_names):
    print(f"{metric_name}: {evaluation_results[i]}")

2023-10-04 18:26:38.469940: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


422/422 [==============================] - 12s 22ms/step - loss: 0.0888 - accuracy: 0.9753
Evaluation Metrics:
loss: 0.08875712752342224
accuracy: 0.9752777814865112


In [25]:
evaluation_results2 = model2.evaluate(xtest, y_test_true, batch_size=2048)
# Print the evaluation metrics
print("Evaluation Metrics:")
for i, metric_name in enumerate(model2.metrics_names):
    print(f"{metric_name}: {evaluation_results2[i]}")

422/422 [==============================] - 9s 22ms/step - loss: inf
Evaluation Metrics:


TypeError: 'float' object is not subscriptable

In [27]:
metric_name


'loss'